In [1]:
%%bash
ls

1.dataSetup.ipynb
2.CNNLSTM.ipynb
3. kerasOnlyExecution.ipynb
4. Subject Specific Classification.ipynb
5. Autoencoder.ipynb
__pycache__
history.pkl
history3.pkl
history4.pkl
scratch.ipynb
tf_mapper.py
tfrecords.py


In [1]:
import tensorflow as tf
# tf.enable_eager_execution()
import tf_mapper as tfm

def read_tfrecord(example):
    features = { \
                'data':  tf.io.FixedLenFeature([1500*3], tf.float32,),\
                'on_off':  tf.io.FixedLenFeature([1], tf.int64,),\
                'dyskinesia':  tf.io.FixedLenFeature([1], tf.int64,),
                'measurement_id':  tf.io.FixedLenFeature([1], tf.int64,),\
                'tremor':  tf.io.FixedLenFeature([1], tf.int64,), \
                'age':  tf.io.FixedLenFeature([1], tf.int64,), \
                "subjects": tf.io.FixedLenFeature([1], tf.int64), \
                "gender": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_PartI_Total": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_PartII_Total": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.1": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.2": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.3": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.4": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.5": tf.io.FixedLenFeature([1], tf.int64), \
                "UPDRS_4.6": tf.io.FixedLenFeature([1], tf.int64)
               }

    example = tf.io.parse_single_example(example, features)
    return example
def map_example_to_simple_with_mid(example):
    data = example['data']
    data = tf.reshape(data, (1500,3))
    return data, example['measurement_id']
def tf_is_in_set(a, b):
    return tf.reduce_sum(tf.cast(tf.equal(b, a), tf.int64)) >= 1
#https://github.com/matthew-brett/transforms3d/blob/master/transforms3d/axangles.py
def tfaxangle2mat(x, y, z, angle, is_normalized=False):
#     x, y, z = axis
    if not is_normalized:
        n = tf.math.sqrt(x*x + y*y + z*z)
        x = x/n
        y = y/n
        z = z/n
    c = tf.math.cos(angle); s = tf.math.sin(angle); C = 1-c
    xs = x*s;   ys = y*s;   zs = z*s
    xC = x*C;   yC = y*C;   zC = z*C
    xyC = x*yC; yzC = y*zC; zxC = z*xC
    return tf.reshape(tf.concat([
             x*xC+c,   xyC-zs,   zxC+ys ,
             xyC+zs,   y*yC+c,   yzC-xs ,
             zxC-ys,   yzC+xs,   z*zC+c ], axis=-1), (3,3))
std = 1/4 #allow deviation from real rotation with pi/4 std
def map_example_to_simple(example):
    data = example["data"]
    data = tf.reshape(data, (1500,3))
    update_matrix = tfaxangle2mat(tf.constant(0.0), tf.constant(0.0), tf.constant(1.0), tf.random.normal((1,)) * tf.constant(3.14*std))
    update_matrix = update_matrix @ tfaxangle2mat(tf.constant(0.0), tf.constant(1.0), tf.constant(0.0), tf.random.normal((1,)) * tf.constant(3.14*std))
    update_matrix = update_matrix @ tfaxangle2mat(tf.constant(1.0), tf.constant(0.0), tf.constant(0.0), tf.random.normal((1,)) * tf.constant(3.14*std))
    data = data @ update_matrix
    return data, data
def get_batched_dataset(filenames, batch_size, m_ids, max_queue_size=10,  n_process=4, is_train=False):
    option_no_order = tf.data.Options()
    option_no_order.experimental_deterministic = False

    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    dataset = dataset.interleave(tf.data.TFRecordDataset, cycle_length=1, num_parallel_calls=1)

    dataset = dataset.map(read_tfrecord, num_parallel_calls=n_process)
    dataset = dataset.filter(lambda example: tf_is_in_set(example["measurement_id"][0], tf.constant(m_ids, dtype=tf.int64)))
    dataset = dataset.filter(lambda example:  tf.math.logical_not(tf.reduce_any(tf.math.is_nan(example["data"]))))
    dataset = dataset.map(map_example_to_simple)
    dataset = dataset.repeat()
    if is_train:
        dataset = dataset.shuffle(2056)
#     dataset = dataset.batch(batch_size, drop_remainder=True)
    if is_train:
        dataset = dataset.prefetch(max_queue_size)
    else:
        dataset = dataset.prefetch(int(max_queue_size/4)) #store a lot less for the other sets to avoid wasting memory
    dataset = dataset.batch(batch_size, drop_remainder=True)
    return dataset

/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ms994/miniconda3/envs/keras-redo/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ms994/miniconda3/envs/keras-

In [2]:
labels = tfm.get_labels()

In [3]:
labels.shape

(1858, 5)

In [4]:
from sklearn.model_selection import train_test_split
train_mid, test_mid = train_test_split(labels.measurement_id.unique(), random_state=1)
train_mid, valid_mid = train_test_split(train_mid, random_state=1)
all_mid = sorted(labels.measurement_id)
train_indices  = [all_mid.index(train_m) for train_m in train_mid]
valid_indices  = [all_mid.index(train_m) for train_m in valid_mid]
test_indices  = [all_mid.index(train_m) for train_m in test_mid]

In [5]:
train_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=train_indices, batch_size=128)
valid_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=valid_indices, batch_size=256)
test_data = get_batched_dataset("/n/scratch2/beat_pd_ms_tmp/all_data.tfr", m_ids=test_indices, batch_size=256)

In [ ]:
train_data

In [17]:
num_cnn_layers = 2
num_lstm_layers = 2
dropout = 0.5
lin_h=16*40
lstm_h = 4
inputLayer = tf.keras.layers.Input((1500, 3))
x = inputLayer
x = tf.keras.layers.GaussianNoise(0.01)(x)


for i in range(num_cnn_layers):
    x = tf.keras.layers.Conv1D(2*(i+1)+1, (3,), padding="same")(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.MaxPool1D((2,))(x)
    x = tf.keras.layers.BatchNormalization()(x)

for j in range(num_lstm_layers):
    x = tf.keras.layers.CuDNNLSTM(int(lstm_h/(2**j)), return_sequences=True)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)

x_shape = x.shape
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(lin_h)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(lin_h/2)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(lin_h/4)(x)
x = tf.keras.layers.LeakyReLU(name="encoder_output")(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(lin_h/4)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(lin_h/2)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dense(x_shape[1].value * x_shape[2].value)(x)
x = tf.keras.layers.LeakyReLU()(x)
x = tf.keras.layers.Reshape((x_shape[1].value, x_shape[2].value))(x)

for j in range(num_lstm_layers):
    x = tf.keras.layers.CuDNNLSTM(int(lstm_h/(2**(num_lstm_layers-j))), return_sequences=True)(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
for i in range(num_cnn_layers):
    x = tf.keras.layers.Conv1D(2*num_cnn_layers-2*(i)+1, (3,), padding="same")(x)
    x = tf.keras.layers.LeakyReLU()(x)
    x = tf.keras.layers.UpSampling1D(2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv1D(3, (3,), padding="same")(x)
model = tf.keras.models.Model(inputs=inputLayer, outputs=x)
model.summary()
model.compile("adam", loss="logcosh", metrics=["mean_squared_error"])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1500, 3)           0         
_________________________________________________________________
gaussian_noise_4 (GaussianNo (None, 1500, 3)           0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 1500, 3)           30        
_________________________________________________________________
leaky_re_lu_39 (LeakyReLU)   (None, 1500, 3)           0         
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 750, 3)            0         
_________________________________________________________________
batch_normalization_v1_44 (B (None, 750, 3)            12        
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 750, 5)            50        
__________

In [13]:
import numpy as np

In [14]:
model.predict(np.random.random((1,1500,3)))

array([[[-9.6213407e-05, -1.8305256e-04, -9.4585028e-04],
        [ 3.7469716e-05,  3.9976966e-04, -2.1422752e-03],
        [ 2.2183784e-04,  4.4619126e-04, -2.7612848e-03],
        ...,
        [-1.5470010e-04, -1.9955179e-03, -1.6932752e-03],
        [-8.7287772e-04, -1.6650428e-03, -2.7994625e-04],
        [ 3.4059914e-05, -1.9781585e-04, -5.8310950e-05]]], dtype=float32)

In [11]:
x, y = None, None
with tf.Session() as sess:
    x, y = train_data.take(10).make_one_shot_iterator().get_next()
    y = y.eval()
    x = x.eval()

In [15]:
np.isnan(x).any()

False

In [18]:
earlyStopping = tf.keras.callbacks.EarlyStopping(patience=20, verbose=True)
modelCheckpoint = tf.keras.callbacks.ModelCheckpoint("/n/scratch2/ms994/autoEncoderCISPD.h5", save_best_only=True)
history = model.fit(train_data, steps_per_epoch=500, epochs=200, validation_data=valid_data, validation_steps=100, callbacks=[earlyStopping, modelCheckpoint],)

Epoch 1/200
500/500 [==============================] - 106s 213ms/step - loss: 0.0967 - mean_squared_error: 0.2182 - val_loss: 0.0840 - val_mean_squared_error: 0.1830
Epoch 2/200
500/500 [==============================] - 88s 175ms/step - loss: 0.0714 - mean_squared_error: 0.1558 - val_loss: 0.0744 - val_mean_squared_error: 0.1622
Epoch 3/200
500/500 [==============================] - 87s 173ms/step - loss: 0.0647 - mean_squared_error: 0.1406 - val_loss: 0.0645 - val_mean_squared_error: 0.1395
Epoch 4/200
500/500 [==============================] - 88s 176ms/step - loss: 0.0641 - mean_squared_error: 0.1392 - val_loss: 0.0622 - val_mean_squared_error: 0.1347
Epoch 5/200
500/500 [==============================] - 88s 175ms/step - loss: 0.0588 - mean_squared_error: 0.1274 - val_loss: 0.0578 - val_mean_squared_error: 0.1243
Epoch 6/200
500/500 [==============================] - 87s 174ms/step - loss: 0.0584 - mean_squared_error: 0.1264 - val_loss: 0.0564 - val_mean_squared_error: 0.1215
Epo

500/500 [==============================] - 86s 173ms/step - loss: 0.0222 - mean_squared_error: 0.0479 - val_loss: 0.0223 - val_mean_squared_error: 0.0479
Epoch 51/200
500/500 [==============================] - 87s 173ms/step - loss: 0.0221 - mean_squared_error: 0.0477 - val_loss: 0.0216 - val_mean_squared_error: 0.0464
Epoch 52/200
500/500 [==============================] - 87s 174ms/step - loss: 0.0221 - mean_squared_error: 0.0478 - val_loss: 0.0213 - val_mean_squared_error: 0.0457
Epoch 53/200
500/500 [==============================] - 87s 175ms/step - loss: 0.0220 - mean_squared_error: 0.0476 - val_loss: 0.0207 - val_mean_squared_error: 0.0446
Epoch 54/200
500/500 [==============================] - 86s 173ms/step - loss: 0.0223 - mean_squared_error: 0.0481 - val_loss: 0.0212 - val_mean_squared_error: 0.0454
Epoch 55/200
500/500 [==============================] - 87s 174ms/step - loss: 0.0219 - mean_squared_error: 0.0475 - val_loss: 0.0218 - val_mean_squared_error: 0.0467
Epoch 56/20

500/500 [==============================] - 87s 174ms/step - loss: 0.0180 - mean_squared_error: 0.0389 - val_loss: 0.0178 - val_mean_squared_error: 0.0382
Epoch 100/200
500/500 [==============================] - 86s 173ms/step - loss: 0.0180 - mean_squared_error: 0.0389 - val_loss: 0.0171 - val_mean_squared_error: 0.0367
Epoch 101/200
500/500 [==============================] - 86s 172ms/step - loss: 0.0179 - mean_squared_error: 0.0388 - val_loss: 0.0178 - val_mean_squared_error: 0.0382
Epoch 102/200
500/500 [==============================] - 88s 175ms/step - loss: 0.0180 - mean_squared_error: 0.0390 - val_loss: 0.0170 - val_mean_squared_error: 0.0365
Epoch 103/200
500/500 [==============================] - 86s 171ms/step - loss: 0.0181 - mean_squared_error: 0.0393 - val_loss: 0.0176 - val_mean_squared_error: 0.0378
Epoch 104/200
500/500 [==============================] - 86s 172ms/step - loss: 0.0180 - mean_squared_error: 0.0390 - val_loss: 0.0165 - val_mean_squared_error: 0.0354
Epoch 

Epoch 148/200
500/500 [==============================] - 86s 172ms/step - loss: 0.0168 - mean_squared_error: 0.0364 - val_loss: 0.0157 - val_mean_squared_error: 0.0337
Epoch 149/200
500/500 [==============================] - 87s 173ms/step - loss: 0.0166 - mean_squared_error: 0.0360 - val_loss: 0.0154 - val_mean_squared_error: 0.0331
Epoch 150/200
500/500 [==============================] - 86s 171ms/step - loss: 0.0173 - mean_squared_error: 0.0374 - val_loss: 0.0171 - val_mean_squared_error: 0.0366
Epoch 151/200
500/500 [==============================] - 87s 173ms/step - loss: 0.0170 - mean_squared_error: 0.0369 - val_loss: 0.0169 - val_mean_squared_error: 0.0362
Epoch 152/200
500/500 [==============================] - 87s 173ms/step - loss: 0.0169 - mean_squared_error: 0.0366 - val_loss: 0.0157 - val_mean_squared_error: 0.0336
Epoch 153/200
500/500 [==============================] - 87s 173ms/step - loss: 0.0169 - mean_squared_error: 0.0366 - val_loss: 0.0155 - val_mean_squared_error:

Epoch 197/200
500/500 [==============================] - 86s 172ms/step - loss: 0.0159 - mean_squared_error: 0.0345 - val_loss: 0.0158 - val_mean_squared_error: 0.0338
Epoch 198/200
500/500 [==============================] - 86s 171ms/step - loss: 0.0159 - mean_squared_error: 0.0344 - val_loss: 0.0147 - val_mean_squared_error: 0.0315
Epoch 199/200
500/500 [==============================] - 87s 173ms/step - loss: 0.0159 - mean_squared_error: 0.0344 - val_loss: 0.0146 - val_mean_squared_error: 0.0315
Epoch 200/200
500/500 [==============================] - 87s 173ms/step - loss: 0.0161 - mean_squared_error: 0.0348 - val_loss: 0.0151 - val_mean_squared_error: 0.0325
